# Home Credit Modeling

In [149]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
import pickle
import gc

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 78.9 ms


#### Load the data

In [183]:
df = pd.read_csv(path + "train.csv")

time: 36.6 s


In [184]:
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 455 ms


#### Select features

In [185]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

462

time: 3min 23s


In [186]:
var_imp = pd.DataFrame({"Feature": df.columns, "Importance": clf.feature_importances_})[["Feature", "Importance"]].sort_values("Importance", ascending=False)

time: 16.8 ms


#### View important features

In [187]:
var_imp.head(20)

,Feature,Importance
32,EXT_SOURCE_3,386
31,EXT_SOURCE_2,379
367,AMT_CREDIT_DIV_AMT_ANNUITY,277
407,EXT_SOURCE_1_DIV_DAYS_BIRTH,248
8,DAYS_BIRTH,203
409,AVG_AGG_SYNTHETIC_TARGET,198
395,DAYS_REGISTRATION_DIV_DAYS_LAST_PHONE_CHANGE,186
350,MIN_DAYS_ENTRY_PAYMENT_DIFF_DAYS_INSTALMENT_12M,180
364,AMT_CREDIT_DIV_AMT_GOODS_PRICE,171
11,DAYS_ID_PUBLISH,165


time: 21.3 ms


#### LightGBM CV

In [ ]:
params = {"n_estimators": 20000, 
          "num_leaves": 500, 
          "min_data_in_leaf": 1000,
          "learning_rate": 0.005, 
          "bagging_fraction": 0.5, 
          "bagging_freq": 1, 
          "feature_fraction": 0.5, 
          "lambda_l2": 1.2}

lgb_data = lgb.Dataset(data=df, 
                       label=y)

cv_result = lgb.cv(params=params, 
                   train_set=lgb_data, 
                   nfold=5, 
                   metrics="auc", 
                   early_stopping_rounds=200, 
                   stratified=True, 
                   shuffle=True, 
                   verbose_eval=100, 
                   show_stdv=True, 
                   seed=2357)

cv_result = pd.DataFrame(cv_result)

/Users/dsaxton/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's auc: 0.766751 + 0.00198604
[200]	cv_agg's auc: 0.770629 + 0.00192756
[300]	cv_agg's auc: 0.774286 + 0.00188278
[400]	cv_agg's auc: 0.777267 + 0.00189604
[500]	cv_agg's auc: 0.779887 + 0.00179658
[600]	cv_agg's auc: 0.782038 + 0.00171513
[700]	cv_agg's auc: 0.783698 + 0.00170026
[800]	cv_agg's auc: 0.785083 + 0.00169543
[900]	cv_agg's auc: 0.786334 + 0.00175759
[1000]	cv_agg's auc: 0.787358 + 0.0017759
[1100]	cv_agg's auc: 0.788115 + 0.00187658
[1200]	cv_agg's auc: 0.788781 + 0.00191058
[1300]	cv_agg's auc: 0.789424 + 0.0019506
[1400]	cv_agg's auc: 0.789955 + 0.00195543
[1500]	cv_agg's auc: 0.790368 + 0.00199909
[1600]	cv_agg's auc: 0.790743 + 0.00202863
[1700]	cv_agg's auc: 0.791074 + 0.00200372
[1800]	cv_agg's auc: 0.79137 + 0.0019819
[1900]	cv_agg's auc: 0.791605 + 0.00206319
[2000]	cv_agg's auc: 0.791788 + 0.00203985
[2100]	cv_agg's auc: 0.79194 + 0.00205392
[2200]	cv_agg's auc: 0.79208 + 0.00204907
[2300]	cv_agg's auc: 0.792201 + 0.00207151
[2400]	cv_agg's auc: 0.792

In [ ]:
cv_result.tail()

CV mean: 0.792743

```
params = {"n_estimators": 3183, 
          "num_leaves": 500, 
          "min_data_in_leaf": 1000,
          "learning_rate": 0.005, 
          "bagging_fraction": 0.5, 
          "bagging_freq": 1, 
          "feature_fraction": 0.5, 
          "lambda_l2": 1}
```

#### Fit chosen model

In [176]:
lgb_model = lgb.LGBMClassifier(n_estimators=2767, 
                         num_leaves=500, 
                         min_data_in_leaf=1000,
                         learning_rate=0.005, 
                         bagging_fraction=0.5, 
                         feature_fraction=0.5, 
                         lambda_l2=0.8)

lgb_model.fit(df[lgb_cols], y)

LGBMClassifier(bagging_fraction=0.5, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, feature_fraction=0.5, lambda_l2=0.8,
        learning_rate=0.005, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=1000, min_split_gain=0.0,
        n_estimators=2767, n_jobs=-1, num_leaves=500, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

time: 17min 56s


#### Score test data and generate submission

In [177]:
df = pd.read_csv(path + "test.csv")

time: 4.12 s


In [178]:
submission = pd.DataFrame({"SK_ID_CURR": df["SK_ID_CURR"], 
                           "TARGET": lgb_model.predict_proba(df[lgb_cols])[:,1]})

submission.head()

,SK_ID_CURR,TARGET
0,100001,0.043615
1,100005,0.136380
2,100013,0.023205
3,100028,0.035034
4,100038,0.222979


time: 8.85 s


In [179]:
submission.to_csv(path + "submission.csv", index=False)

time: 178 ms
